In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Project_576/
!ls -la

/content/drive/My Drive/Colab Notebooks/Project_576
total 1000890
-rw------- 1 root root 145975841 Apr 27 00:33 decoder_nmt_13.pkl
-rw------- 1 root root 145975857 Apr 26 22:42 decoder_nmt_LR1e-5_2ndep.pkl
-rw------- 1 root root 145975849 Apr 26 10:54 decoder_nmt_LR1e-5_3ep.pkl
-rw------- 1 root root 145975857 Apr 26 04:59 decoder_nmt.pkl
-rw------- 1 root root  57338758 Apr 27 00:33 encoder_nmt_13.pkl
-rw------- 1 root root  57338758 Apr 26 22:42 encoder_nmt_LR1e-5_2ndep.pkl
-rw------- 1 root root  57338758 Apr 26 10:54 encoder_nmt_LR1e-5_3ep.pkl
-rw------- 1 root root  57338758 Apr 26 04:59 encoder_nmt.pkl
-rw------- 1 root root   3395237 Apr 26 22:56 Flickr8k.token.txt
-rw------- 1 root root  45442087 Apr 26 22:48 hindencorp05.export.gz
-rw------- 1 root root  72209027 Apr 26 22:56 hindencorp05.plaintext
-rw------- 1 root root  23899723 Apr 26 22:48 hindencorp05.plaintext.gz
-rw------- 1 root root  66672230 Apr 26 22:56 hindencorp05.txt
-rw------- 1 root root     24933 Apr 27 05:22 

In [0]:
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-097C-0000-0023-625F-0{/README.txt,/hindencorp05.export.gz,/hindencorp05.plaintext.gz}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4472  100  4472    0     0   3717      0  0:00:01  0:00:01 --:--:--  3720
100 43.3M  100 43.3M    0     0  17.2M      0  0:00:02  0:00:02 --:--:-- 78.3M
100 22.7M  100 22.7M    0     0  11.6M      0  0:00:01  0:00:01 --:--:-- 83.7M


In [0]:
!gunzip -k hindencorp05.plaintext.gz

gzip: hindencorp05.plaintext already exists; do you wish to overwrite (y or n)? y


In [0]:
!cut -f 4- hindencorp05.plaintext > hindencorp05.txt

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    table = str.maketrans('', '', string.punctuation)
    s = s.translate(table)
    s = re.sub(r"[.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('hindencorp05.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 15

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'hin', False)
print(random.choice(pairs))

Reading lines...
Read 273885 sentence pairs
Trimmed to 182030 sentence pairs
Counting words...
Counted words:
eng 54452
hin 70097
['export multiple   ', 'अनेक निर्यात   m']


In [0]:
%matplotlib inline
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
%matplotlib inline
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
%matplotlib inline
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    %matplotlib inline
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
%matplotlib inline
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    %matplotlib inline

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
%matplotlib inline
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
random.choice(pairs)

['banning song', 'गाना प्रतिबंधित कर रहा है']

In [0]:
len(pairs)

182030

In [0]:
%matplotlib inline
def trainIters(encoder, decoder, n_epochs, learning_rate, print_every=50000, plot_every=50000):
    %matplotlib inline
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pairs[i]) for i in range(len(pairs))]
    criterion = nn.NLLLoss()
    
    n_iters = len(pairs)
    for e in range(1, n_epochs + 1):
        for iter in range(1, len(pairs) + 1):
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    %matplotlib inline
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    %matplotlib inline
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=5):
    from nltk.translate.bleu_score import sentence_bleu
    %matplotlib inline
    for i in range(n):
        pair = random.choice(pairs)
        print('English: ', pair[0])
        print('Actual: ', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('Predicted: ', output_sentence)
        act = pair[1].split()
        pred = output_sentence.split()
        print('Cumulative 1-gram: %f' % sentence_bleu([act], pred, weights=(1, 0, 0, 0)))
        print('Cumulative 2-gram: %f' % sentence_bleu([act], pred, weights=(0.5, 0.5, 0, 0)))
        print('Cumulative 3-gram: %f' % sentence_bleu([act], pred, weights=(0.33, 0.33, 0.33, 0)))
        print('Cumulative 4-gram: %f' % sentence_bleu([act], pred, weights=(0.25, 0.25, 0.25, 0.25)))
        score = sentence_bleu([act], pred)
        print(score)
        print('')

In [0]:
%matplotlib inline
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

encoder1.load_state_dict(torch.load('encoder_nmt_LR1e-5_2ndep.pkl'))
attn_decoder1.load_state_dict(torch.load('decoder_nmt_LR1e-5_2ndep.pkl'))

#trainIters(encoder1, attn_decoder1, 1, print_every=20000, learning_rate=0.00001)


torch.save(encoder1.state_dict(),'encoder_nmt_13.pkl')
torch.save(attn_decoder1.state_dict(),'decoder_nmt_13.pkl')

<All keys matched successfully>

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

English:  you know what grannies do ”
Actual:  तुम्हें पता है दादियाँ क्या करती हैं ”
Predicted:  क्या आप हैं  ” 
Cumulative 1-gram: 0.275910
Cumulative 2-gram: 0.183940
Cumulative 3-gram: 0.232823
Cumulative 4-gram: 0.260130
0.2601300475114444

English:  compact layout
Actual:  компактний вигляд
Predicted:  kompakt 
Cumulative 1-gram: 0.000000
Cumulative 2-gram: 0.000000
Cumulative 3-gram: 0.000000
Cumulative 4-gram: 0.000000
0

English:   have a disability
Actual:   विकलांगता है
Predicted:  एक है 
Cumulative 1-gram: 0.500000
Cumulative 2-gram: 0.707107
Cumulative 3-gram: 0.795536
Cumulative 4-gram: 0.840896
0.8408964152537145

English:  saaransh
Actual:  सारांश 1984 फ़िल्म
Predicted:  सारांश 
Cumulative 1-gram: 0.135335
Cumulative 2-gram: 0.135335
Cumulative 3-gram: 0.135335
Cumulative 4-gram: 0.135335
0.1353352832366127

English:  louisiana
Actual:  लुईज़ियाना
Predicted:  लुईज़ियाना 
Cumulative 1-gram: 1.000000
Cumulative 2-gram: 1.000000
Cumulative 3-gram: 1.000000
Cumulative 4-gr

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [0]:
#Flickr8 Dataset Img Captioning 

sent1 = " Two dogs are running together through the grass "
sent2 = " man is on a big wave"
sent3 = " man in a helmet rides a bike prediction"
#sent4 = " man on a surfboard "
sent5 = "woman and a woman are smiling and smiling"
sent6 = "group of people are sitting on a bench"
sent7 = " football player in a red uniform is running "
sent8 = "man in a blue shirt and blue shorts is on the street"
sent9 = "two brown dogs are playing in the snow"




output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
#output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())
output_words9, attentions9 = evaluate(encoder1, attn_decoder1, sent9.lower())

output_sentence1 = ' '.join(output_words1)
output_sentence2 = ' '.join(output_words2)
output_sentence3 = ' '.join(output_words3)
#output_sentence4 = ' '.join(output_words4)
output_sentence5 = ' '.join(output_words5)
output_sentence6 = ' '.join(output_words6)
output_sentence7 = ' '.join(output_words7)
output_sentence8 = ' '.join(output_words8)
output_sentence9 = ' '.join(output_words9)


print('Predicted1: ', output_sentence1)
print('Predicted2: ', output_sentence2)
print('Predicted3: ', output_sentence3)
#print('Predicted4: ', output_sentence4)
print('Predicted5: ', output_sentence5)
print('Predicted6: ', output_sentence6)
print('Predicted7: ', output_sentence7)
print('Predicted8: ', output_sentence8)
print('Predicted9: ', output_sentence9)

Predicted1:  दो  
Predicted2:  यह पर पर पर पर पर पर पर पर पर पर पर पर पर पर
Predicted3:  के में 
Predicted5:  और एक एक एक एक एक एक एक एक एक एक एक एक एक एक
Predicted6:  लोग लोग लोग लोग के लोग का एक 
Predicted7:  लाल रंग में एक रंग 
Predicted8:  एक सड़क पर आधारित 
Predicted9:  दो देश में 


In [0]:
sent1 = " Two dogs are running together through the grass "
sent2 = " man is on a big wave"
sent3 = " man in a helmet rides a bike prediction"
#sent4 = " man on a surfboard "
sent5 = "woman and a woman are smiling and smiling"
sent6 = "group of people are sitting on a bench"
sent7 = " football player in a red uniform is running "
sent8 = "man in a blue shirt and blue shorts is on the street"
sent9 = "two brown dogs are playing in the snow"

In [0]:
#Hindencorp sentences



sent1 = "You may have noticed it"
sent2 = "Place of Birth"
sent3 = "Once that trust was built"
sent4 = "Woolly rhinoceros"
sent5 = "Flag of Kazakhstan"
sent6 = "But male chauvinism was its blind side"
sent7 = "for the world in general"
sent8 = "Oriya language"




output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())

output_sentence1 = ' '.join(output_words1)
output_sentence2 = ' '.join(output_words2)
output_sentence3 = ' '.join(output_words3)
output_sentence4 = ' '.join(output_words4)
output_sentence5 = ' '.join(output_words5)
output_sentence6 = ' '.join(output_words6)
output_sentence7 = ' '.join(output_words7)
output_sentence8 = ' '.join(output_words8)


print('Predicted1: ', output_sentence1)
print('Predicted2: ', output_sentence2)
print('Predicted3: ', output_sentence3)
print('Predicted4: ', output_sentence4)
print('Predicted5: ', output_sentence5)
print('Predicted6: ', output_sentence6)
print('Predicted7: ', output_sentence7)
print('Predicted8: ', output_sentence8)



Predicted1:  आप आप आप आप आप आप आप आप आप आप आप आप आप आप आप
Predicted2:  जन्म की 
Predicted3:  यह था 
Predicted4:  बालदार 
Predicted5:  का ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज ध्वज
Predicted6:  लेकिन वे ही सेनाओं 
Predicted7:  दुनिया के लिए में 
Predicted8:  भाषा 
